In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from secret import username, password

# EXTRACT

Download movies and names CSV files from https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset and place in a folder named data

In [2]:
# Study data files
imdb_movies_filepath = "data/IMDb movies.csv"
imdb_names_filepath = "data/IMDb names.csv"

In [3]:
# Read the data into Pandas DataFrame
imdb_movies = pd.read_csv(imdb_movies_filepath)
imdb_names = pd.read_csv(imdb_names_filepath)

# TRANSFORM

## PERSON FILE

In [4]:
#Look at imported names CSV
imdb_names.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,birth_year,date_of_birth,place_of_birth,death_details,death_year,date_of_death,place_of_death,reason_of_death,spouses,divorces,spouses_with_children,children,primary_profession,known_for_titles
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899.0,1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987.0,1987-06-22,"Los Angeles, California, USA",pneumonia,2,0,1,2,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0043044"
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924.0,1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014.0,2014-08-12,"New York City, New York, USA",stroke,2,1,2,3,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934.0,1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,NaN,4,3,1,1,"actress,soundtrack,producer","tt0054452,tt0059956,tt0057345,tt0049189"
3,nm0000004,John Belushi,John Adam Belushi,173.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949.0,1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982.0,1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,1,0,0,0,"actor,writer,soundtrack","tt0078723,tt0072562,tt0080455,tt0077975"
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918.0,1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden",2007.0,2007-07-30,"Fårö, Gotlands län, Sweden",NaN,5,4,5,8,"writer,director,actor","tt0050976,tt0083922,tt0069467,tt0050986"


### person table

In [5]:
# Return only wanted columns for person table
name_columns = ['imdb_name_id', 'name', 'birth_name', 'birth_year', 'death_year', 'height']
new_name_df = imdb_names[name_columns].copy()

# Rename the column headers to match ERD
transform_name_df = new_name_df.rename(columns={'imdb_name_id': 'person_id',
                                                 'name': 'person_name'})

# Drop duplicate people with same name
final_name = transform_name_df.drop_duplicates('person_name')

final_name.head()

,person_id,person_name,birth_name,birth_year,death_year,height
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,1899.0,1987.0,177.0
1,nm0000002,Lauren Bacall,Betty Joan Perske,1924.0,2014.0,174.0
2,nm0000003,Brigitte Bardot,Brigitte Bardot,1934.0,NaN,166.0
3,nm0000004,John Belushi,John Adam Belushi,1949.0,1982.0,173.0
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,1918.0,2007.0,179.0


In [6]:
#Inspect name table for column type
final_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173127 entries, 0 to 175714
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   person_id    173127 non-null  object 
 1   person_name  173127 non-null  object 
 2   birth_name   173127 non-null  object 
 3   birth_year   74487 non-null   float64
 4   death_year   26639 non-null   float64
 5   height       29784 non-null   float64
dtypes: float64(3), object(3)
memory usage: 9.2+ MB


## MOVIES FILE

In [7]:
# Inspect movies CSV
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0


### job_title table

In [8]:
#Create lists for job title id and job title
job_title_id = ['jt001', 'jt002', 'jt003']
job_title = ['actor', 'director', 'writer']

#Convert list to DataFrame
job_title_df = pd.DataFrame(list(zip(job_title_id, job_title)), 
               columns =['job_title_id', 'job_title'])
job_title_df.head()

,job_title_id,job_title
0,jt001,actor
1,jt002,director
2,jt003,writer


### movie_person_title_junction table

#### Directors

In [9]:
# Return only directors and movie title id's
director_df = imdb_movies[['imdb_title_id', 'director']].copy()
director_df.head()

,imdb_title_id,director
0,tt0000574,Charles Tait
1,tt0001892,Urban Gad
2,tt0002101,Charles L. Gaskill
3,tt0002130,"Francesco Bertolini, Adolfo Padovan"
4,tt0002199,Sidney Olcott


In [10]:
# Rename the column headers
transform_director_df = director_df.rename(columns={'imdb_title_id': 'movie_id',
                                                     'director': 'person_name'})

# Split person_name column by comma and insert it as new row
new_director_df = transform_director_df.assign(person_name=transform_director_df.person_name.str.split(', ')).explode('person_name')

# Add job title id to all rows
new_director_df['job_title_id'] ='jt002'
new_director_df.head()

,movie_id,person_name,job_title_id
0,tt0000574,Charles Tait,jt002
1,tt0001892,Urban Gad,jt002
2,tt0002101,Charles L. Gaskill,jt002
3,tt0002130,Francesco Bertolini,jt002
3,tt0002130,Adolfo Padovan,jt002


In [11]:
#Map person name to name DataFrame and add column person_id
new_director_df['person_id'] = new_director_df['person_name'].map(final_name.set_index('person_name')['person_id'])

#Drop person name column to form first part of movie_person_title_junction table
final_director = new_director_df.drop(columns='person_name')
final_director.head()

,movie_id,job_title_id,person_id
0,tt0000574,jt002,nm0846879
1,tt0001892,jt002,NaN
2,tt0002101,jt002,nm0309130
3,tt0002130,jt002,nm0078205
3,tt0002130,jt002,nm0655824


#### Writers

In [12]:
# Return only writers and movie title id's
writer_df = imdb_movies[['imdb_title_id', 'writer']].copy()
writer_df.head()

,imdb_title_id,writer
0,tt0000574,Charles Tait
1,tt0001892,"Urban Gad, Gebhard Schätzler-Perasini"
2,tt0002101,Victorien Sardou
3,tt0002130,Dante Alighieri
4,tt0002199,Gene Gauntier


In [13]:
# Rename the column headers
transform_writer_df = writer_df.rename(columns={'imdb_title_id': 'movie_id',
                                                     'writer': 'person_name'})

# Split person_name column by comma and insert it as new row
new_writer_df = transform_writer_df.assign(person_name=transform_writer_df.person_name.str.split(', ')).explode('person_name')

# Add job title id to all rows
new_writer_df['job_title_id'] ='jt003'
new_writer_df.head()

,movie_id,person_name,job_title_id
0,tt0000574,Charles Tait,jt003
1,tt0001892,Urban Gad,jt003
1,tt0001892,Gebhard Schätzler-Perasini,jt003
2,tt0002101,Victorien Sardou,jt003
3,tt0002130,Dante Alighieri,jt003


In [14]:
#Map person name to name DataFrame and add column person_id
new_writer_df['person_id'] = new_writer_df['person_name'].map(final_name.set_index('person_name')['person_id'])

#Drop person name column to form second part of movie_person_title_junction table
final_writer = new_writer_df.drop(columns='person_name')
final_writer.head()

,movie_id,job_title_id,person_id
0,tt0000574,jt003,nm0846879
1,tt0001892,jt003,NaN
1,tt0001892,jt003,NaN
2,tt0002101,jt003,nm0765026
3,tt0002130,jt003,nm0019604


#### Actors

In [15]:
# Return only actors and movie title id's
actor_df = imdb_movies[['imdb_title_id', 'actors']].copy()
actor_df.head()

,imdb_title_id,actors
0,tt0000574,"Elizabeth Tait, John Tait, Norman Campbell, Be..."
1,tt0001892,"Asta Nielsen, Valdemar Psilander, Gunnar Helse..."
2,tt0002101,"Helen Gardner, Pearl Sindelar, Miss Fielding, ..."
3,tt0002130,"Salvatore Papa, Arturo Pirovano, Giuseppe de L..."
4,tt0002199,"R. Henderson Bland, Percy Dyer, Gene Gauntier,..."


In [16]:
# Rename the column headers
transform_actor_df = actor_df.rename(columns={'imdb_title_id': 'movie_id',
                                                     'actors': 'person_name'})

# Split person_name column by comma and insert it as new row
new_actor_df = transform_actor_df.assign(person_name=transform_actor_df.person_name.str.split(', ')).explode('person_name')

# Add job title id to all rows
new_actor_df['job_title_id'] ='jt001'
new_actor_df.head()

,movie_id,person_name,job_title_id
0,tt0000574,Elizabeth Tait,jt001
0,tt0000574,John Tait,jt001
0,tt0000574,Norman Campbell,jt001
0,tt0000574,Bella Cola,jt001
0,tt0000574,Will Coyne,jt001


In [17]:
#Map person name to name DataFrame and add column person_id
new_actor_df['person_id'] = new_actor_df['person_name'].map(final_name.set_index('person_name')['person_id'])

#Drop person name column to form last part of movie_person_title_junction table
final_actor = new_actor_df.drop(columns='person_name')
final_actor.head()

,movie_id,job_title_id,person_id
0,tt0000574,jt001,nm0846887
0,tt0000574,jt001,nm0846894
0,tt0000574,jt001,nm3002376
0,tt0000574,jt001,nm0170118
0,tt0000574,jt001,NaN


In [18]:
# Combine director, writer, actor DataFrames
combined_roles_df = pd.concat([final_director, final_writer, final_actor], axis=0)

# Drop rows with no person_id
junction_df = combined_roles_df.dropna(subset=['person_id'])
junction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766171 entries, 0 to 81272
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      766171 non-null  object
 1   job_title_id  766171 non-null  object
 2   person_id     766171 non-null  object
dtypes: object(3)
memory usage: 23.4+ MB


In [19]:
# Create unique id with all 3 columns as name
junction_df.loc[:,'unique_id']=junction_df.movie_id.astype(str) + '_' +\
    junction_df.job_title_id.astype(str) + '_' + junction_df.person_id.astype(str)

# Drop duplicates
junction_df2=junction_df.drop_duplicates("unique_id")

# Drop unique_id column
final_junction = junction_df2.drop(columns='unique_id')
final_junction.info()

C:\Users\track\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\track\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<class 'pandas.core.frame.DataFrame'>
Int64Index: 761109 entries, 0 to 81272
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      761109 non-null  object
 1   job_title_id  761109 non-null  object
 2   person_id     761109 non-null  object
dtypes: object(3)
memory usage: 23.2+ MB


In [20]:
final_mpt_junction = final_junction[['movie_id', 'person_id', 'job_title_id']]
final_mpt_junction.head()

,movie_id,person_id,job_title_id
0,tt0000574,nm0846879,jt002
2,tt0002101,nm0309130,jt002
3,tt0002130,nm0078205,jt002
3,tt0002130,nm0655824,jt002
4,tt0002199,nm0646058,jt002


### country_origin table

In [21]:
# Dropping columns and focus on language and country
language_country_df = imdb_movies[['imdb_title_id', 'country', 'language']]
language_country_df.head()

,imdb_title_id,country,language
0,tt0000574,Australia,NaN
1,tt0001892,"Germany, Denmark",NaN
2,tt0002101,USA,English
3,tt0002130,Italy,Italian
4,tt0002199,USA,English


In [22]:
# get the list of unique country names
movie_df_temp1 = language_country_df.set_index('imdb_title_id').country.str.split(', ', expand=True).stack()
uniq_country = movie_df_temp1.unique()
uniq_country.size

190

In [23]:
# create a dataframe 'country_origin_df' with 'country_id' and 'country_name'
temp1 = pd.DataFrame(data=[np.arange(191), uniq_country])
country_origin_df = temp1.transpose()
country_origin_df.columns = ['country_id','country_name']
country_origin_df

,country_id,country_name
0,0,Australia
1,1,Germany
2,2,Denmark
3,3,USA
4,4,Italy
...,...,...
186,186,Guadeloupe
187,187,Malawi
188,188,Holy See (Vatican City State)
189,189,Oman


In [24]:
# Convert country_id column to int
country_origin_df['country_id']=country_origin_df['country_id'].astype(int)
country_origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_id    191 non-null    int32 
 1   country_name  190 non-null    object
dtypes: int32(1), object(1)
memory usage: 2.4+ KB


### movie_country_junction_table

In [25]:
# get the list of unique movie IDs
language_country_df.nunique()

imdb_title_id    81273
country           4632
language          4251
dtype: int64

In [26]:
# get the title_id and country columns from original df
movie_country_df = imdb_movies[['imdb_title_id', 'country']]
movie_country_df.head()

,imdb_title_id,country
0,tt0000574,Australia
1,tt0001892,"Germany, Denmark"
2,tt0002101,USA
3,tt0002130,Italy
4,tt0002199,USA


In [27]:
# rename title id column
mc_df = movie_country_df.rename(columns={'imdb_title_id': 'movie_id'})
mc_df.head()

,movie_id,country
0,tt0000574,Australia
1,tt0001892,"Germany, Denmark"
2,tt0002101,USA
3,tt0002130,Italy
4,tt0002199,USA


In [28]:
# make new column that has only one country per row
movie_df_temp1 = mc_df.assign(country_new=mc_df.country.str.split(', ')).explode('country_new')
movie_df_temp1.head()

,movie_id,country,country_new
0,tt0000574,Australia,Australia
1,tt0001892,"Germany, Denmark",Germany
1,tt0001892,"Germany, Denmark",Denmark
2,tt0002101,USA,USA
3,tt0002130,Italy,Italy


In [29]:
# map out the country_id column with the corresponding country_id from the country_origin_df
movie_df_temp1['country_id'] = movie_df_temp1['country_new'].map(country_origin_df.set_index('country_name')['country_id'])
movie_df_temp1.head()

,movie_id,country,country_new,country_id
0,tt0000574,Australia,Australia,0
1,tt0001892,"Germany, Denmark",Germany,1
1,tt0001892,"Germany, Denmark",Denmark,2
2,tt0002101,USA,USA,3
3,tt0002130,Italy,Italy,4


In [30]:
# drop unnecessary columns
movie_country_junction_df =movie_df_temp1.drop(columns=['country_new','country'])
movie_country_junction_df.head()

,movie_id,country_id
0,tt0000574,0
1,tt0001892,1
1,tt0001892,2
2,tt0002101,3
3,tt0002130,4


### language table

In [31]:
# now do the same with language
language_country_df.language

0              NaN
1              NaN
2          English
3          Italian
4          English
           ...    
81268       Telugu
81269    Malayalam
81270    Malayalam
81271    Malayalam
81272      Turkish
Name: language, Length: 81273, dtype: object

In [32]:
movie_df_temp2 = language_country_df.set_index('imdb_title_id').language.str.split(', ', expand=True).stack()
movie_df_temp2

imdb_title_id   
tt0002101      0      English
tt0002130      0      Italian
tt0002199      0      English
tt0002423      0       German
tt0002445      0      Italian
                      ...    
tt9903716      0       Telugu
tt9905412      0    Malayalam
tt9905462      0    Malayalam
tt9911774      0    Malayalam
tt9914286      0      Turkish
Length: 103728, dtype: object

In [33]:
uniq_language = movie_df_temp2.unique()
uniq_language.size

264

In [34]:
# create a dataframe 'language_df' with 'language_id' and 'language'
temp2 = pd.DataFrame(data=[np.arange(265), uniq_language])
language_df = temp2.transpose()
language_df.columns = ['language_id','language']
language_df

,language_id,language
0,0,English
1,1,Italian
2,2,German
3,3,Danish
4,4,French
...,...,...
260,260,Bemba
261,261,Wayuu
262,262,Balinese
263,263,Haida


In [35]:
# Convert language_id column to int
language_df['language_id']=language_df['language_id'].astype(int)
language_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   language_id  265 non-null    int32 
 1   language     264 non-null    object
dtypes: int32(1), object(1)
memory usage: 3.2+ KB


### movie_language_junction_table

In [36]:
movie_language_df = imdb_movies[['imdb_title_id', 'language']]
movie_language_df

,imdb_title_id,language
0,tt0000574,NaN
1,tt0001892,NaN
2,tt0002101,English
3,tt0002130,Italian
4,tt0002199,English
...,...,...
81268,tt9903716,Telugu
81269,tt9905412,Malayalam
81270,tt9905462,Malayalam
81271,tt9911774,Malayalam


In [37]:
# rename title id column
ml_df = movie_language_df.rename(columns={'imdb_title_id': 'movie_id'})
ml_df.head()

,movie_id,language
0,tt0000574,NaN
1,tt0001892,NaN
2,tt0002101,English
3,tt0002130,Italian
4,tt0002199,English


In [38]:
#since there are more than one languages for some movies, create a new column 'language_name' with single language in it
movie_df_temp2 = ml_df.assign(language_new=ml_df.language.str.split(', ')).explode('language_new')
movie_df_temp2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104483 entries, 0 to 81272
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      104483 non-null  object
 1   language      103728 non-null  object
 2   language_new  103728 non-null  object
dtypes: object(3)
memory usage: 3.2+ MB


In [39]:
# map out the language_id column with the corresponding language_id from the language_df
movie_df_temp2['language_id'] = movie_df_temp2['language_new'].map(language_df.set_index('language')['language_id'])
movie_df_temp2.head()

,movie_id,language,language_new,language_id
0,tt0000574,NaN,NaN,264
1,tt0001892,NaN,NaN,264
2,tt0002101,English,English,0
3,tt0002130,Italian,Italian,1
4,tt0002199,English,English,0


In [40]:
# drop unnecessary columns
movie_language_junction_df = movie_df_temp2.drop(columns=['language','language_new'])
movie_language_junction_df.head()

,movie_id,language_id
0,tt0000574,264
1,tt0001892,264
2,tt0002101,0
3,tt0002130,1
4,tt0002199,0


In [41]:
# Create unique id with 2 columns as name
movie_language_junction_df.loc[:,'unique_id']= movie_language_junction_df.loc[:,'movie_id'].astype(str) + '_' +\
    movie_language_junction_df.loc[:,'language_id'].astype(str)

# Drop duplicates
movie_language_junction_df.drop_duplicates("unique_id", inplace=True)

# # Drop unique_id column
# movie_language_junction_df.drop(columns='unique_id', inplace=True)
movie_language_junction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104482 entries, 0 to 81272
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   movie_id     104482 non-null  object
 1   language_id  104482 non-null  int32 
 2   unique_id    104482 non-null  object
dtypes: int32(1), object(2)
memory usage: 2.8+ MB


### genre table

In [42]:
# Dropping columns and focus on genre and production company
genre_prodcomp_df = imdb_movies[['imdb_title_id', 'genre', 'production_company']]
genre_prodcomp_df.head()

,imdb_title_id,genre,production_company
0,tt0000574,"Biography, Crime, Drama",J. and N. Tait
1,tt0001892,Drama,Fotorama
2,tt0002101,"Drama, History",Helen Gardner Picture Players
3,tt0002130,"Adventure, Drama, Fantasy",Milano Film
4,tt0002199,"Biography, Drama",Kalem Company


In [43]:
# get the list of unique genre names
movie_df_temp5 = genre_prodcomp_df.set_index('imdb_title_id').genre.str.split(', ', expand=True).stack()
uniq_genre = movie_df_temp5.unique()
uniq_genre.size

26

In [44]:
# create a dataframe 'genre_df' with 'genre_id' and 'genre_name'
temp5 = pd.DataFrame(data=[np.arange(27), uniq_genre])
genre_df = temp5.transpose()
genre_df.columns = ['genre_id','genre_name']
genre_df.head()

,genre_id,genre_name
0,0,Biography
1,1,Crime
2,2,Drama
3,3,History
4,4,Adventure


In [45]:
# Convert genre_id column to int
genre_df['genre_id']=genre_df['genre_id'].astype(int)
genre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   genre_id    27 non-null     int32 
 1   genre_name  26 non-null     object
dtypes: int32(1), object(1)
memory usage: 452.0+ bytes


### movie_genre_junction table

In [46]:
# get the list of unique movie IDs
genre_prodcomp_df.nunique()

imdb_title_id         81273
genre                  1264
production_company    30122
dtype: int64

In [47]:
# get the title_id and genre columns from original df
movie_genre_df = imdb_movies[['imdb_title_id', 'genre']]
movie_genre_df.head()

,imdb_title_id,genre
0,tt0000574,"Biography, Crime, Drama"
1,tt0001892,Drama
2,tt0002101,"Drama, History"
3,tt0002130,"Adventure, Drama, Fantasy"
4,tt0002199,"Biography, Drama"


In [48]:
# rename title id column
mg_df = movie_genre_df.rename(columns={'imdb_title_id': 'movie_id'})
mg_df.head()

,movie_id,genre
0,tt0000574,"Biography, Crime, Drama"
1,tt0001892,Drama
2,tt0002101,"Drama, History"
3,tt0002130,"Adventure, Drama, Fantasy"
4,tt0002199,"Biography, Drama"


In [49]:
# make new column that has only one genre per row
movie_df_temp6 = mg_df.assign(genre_new=mg_df.genre.str.split(', ')).explode('genre_new')
movie_df_temp6.head()

,movie_id,genre,genre_new
0,tt0000574,"Biography, Crime, Drama",Biography
0,tt0000574,"Biography, Crime, Drama",Crime
0,tt0000574,"Biography, Crime, Drama",Drama
1,tt0001892,Drama,Drama
2,tt0002101,"Drama, History",Drama


In [50]:
# map out the genre_id column with the corresponding genre_id from the genre_df
movie_df_temp6['genre_id'] = movie_df_temp6['genre_new'].map(genre_df.set_index('genre_name')['genre_id'])
movie_df_temp6.head()

,movie_id,genre,genre_new,genre_id
0,tt0000574,"Biography, Crime, Drama",Biography,0
0,tt0000574,"Biography, Crime, Drama",Crime,1
0,tt0000574,"Biography, Crime, Drama",Drama,2
1,tt0001892,Drama,Drama,2
2,tt0002101,"Drama, History",Drama,2


In [51]:
# drop unnecessary columns
movie_genre_junction_df =movie_df_temp6.drop(columns=['genre_new','genre'])
movie_genre_junction_df.head()

,movie_id,genre_id
0,tt0000574,0
0,tt0000574,1
0,tt0000574,2
1,tt0001892,2
2,tt0002101,2


In [52]:
# Create unique id with 2 columns as name
movie_genre_junction_df.loc[:,'unique_id']= movie_genre_junction_df.loc[:,'movie_id'].astype(str) + '_' +\
    movie_genre_junction_df.loc[:,'genre_id'].astype(str)

# Drop duplicates
movie_genre_junction_df.drop_duplicates("unique_id", inplace=True)

# # Drop unique_id column
# movie_language_junction_df.drop(columns='unique_id', inplace=True)
movie_genre_junction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166221 entries, 0 to 81272
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   166221 non-null  object
 1   genre_id   166221 non-null  int32 
 2   unique_id  166221 non-null  object
dtypes: int32(1), object(2)
memory usage: 4.4+ MB


### production_company table

In [53]:
# now do the same with production company
genre_prodcomp_df.production_company

0                       J. and N. Tait
1                             Fotorama
2        Helen Gardner Picture Players
3                          Milano Film
4                        Kalem Company
                     ...              
81268              Ekaa Art Production
81269          Thomas Thiruvalla Films
81270                Benzy Productions
81271                 RMCC Productions
81272                      Gizem Ajans
Name: production_company, Length: 81273, dtype: object

In [54]:
movie_df_temp7 = genre_prodcomp_df.set_index('imdb_title_id').production_company
movie_df_temp7

imdb_title_id
tt0000574                   J. and N. Tait
tt0001892                         Fotorama
tt0002101    Helen Gardner Picture Players
tt0002130                      Milano Film
tt0002199                    Kalem Company
                         ...              
tt9903716              Ekaa Art Production
tt9905412          Thomas Thiruvalla Films
tt9905462                Benzy Productions
tt9911774                 RMCC Productions
tt9914286                      Gizem Ajans
Name: production_company, Length: 81273, dtype: object

In [55]:
uniq_prodcomp = movie_df_temp7.unique()
uniq_prodcomp.size

30123

In [56]:
# create a dataframe 'company_df' with 'company_id' and 'company'
temp7 = pd.DataFrame(data=[np.arange(30123), uniq_prodcomp])
company_df = temp7.transpose()
company_df.columns = ['company_id','company_name']
company_df

,company_id,company_name
0,0,J. and N. Tait
1,1,Fotorama
2,2,Helen Gardner Picture Players
3,3,Milano Film
4,4,Kalem Company
...,...,...
30118,30118,Ekaa Art Production
30119,30119,Thomas Thiruvalla Films
30120,30120,Benzy Productions
30121,30121,RMCC Productions


In [57]:
# Create unique id with 2 columns as name
company_df.loc[:,'unique_id']= company_df.loc[:,'company_id'].astype(str) + '_' +\
    company_df.loc[:,'company_name'].astype(str)

# Drop duplicates
company_df.drop_duplicates("unique_id", inplace=True)

# Drop unique_id column
company_df.drop(columns='unique_id', inplace=True)
company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30123 entries, 0 to 30122
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_id    30123 non-null  object
 1   company_name  30122 non-null  object
dtypes: object(2)
memory usage: 706.0+ KB


In [58]:
# Convert company_id column to int
company_df['company_id']=company_df['company_id'].astype(int)
company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30123 entries, 0 to 30122
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_id    30123 non-null  int32 
 1   company_name  30122 non-null  object
dtypes: int32(1), object(1)
memory usage: 588.3+ KB


### movie table

In [59]:
# Return wanted columns for movie table
movie_columns = ['imdb_title_id', 'title', 'year', 'duration', 'description', 'avg_vote', 'votes', 'budget',
                 'usa_gross_income', 'worlwide_gross_income', 'production_company']
new_movie_df = imdb_movies[movie_columns].copy()
new_movie_df.head()

,imdb_title_id,title,year,duration,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,production_company
0,tt0000574,The Story of the Kelly Gang,1906,70,True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,J. and N. Tait
1,tt0001892,Den sorte drøm,1911,53,Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,Fotorama
2,tt0002101,Cleopatra,1912,100,The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,Helen Gardner Picture Players
3,tt0002130,L'Inferno,1911,68,Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,Milano Film
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,60,"An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,Kalem Company


In [60]:
# Rename the column headers
transform_movie_df = new_movie_df.rename(columns={'imdb_title_id': 'movie_id',
                                                 'title': 'movie_title',
                                                 'year': 'year_published',
                                                 'duration': 'movie_duration',
                                                 'votes': 'votes_count',
                                                 'avg_vote': 'votes_avg',
                                                 'production_company': 'company_name'})

# Clean the data by dropping duplicates and setting the index
transform_movie_df.drop_duplicates("movie_id", inplace=True)
transform_movie_df.set_index("movie_id", inplace=True)

transform_movie_df.tail()

,movie_title,year_published,movie_duration,description,votes_avg,votes_count,budget,usa_gross_income,worlwide_gross_income,company_name
movie_id,,,,,,,,,,
tt9903716,Jessie,2019,106,"Set in an abandoned house, the film follows a ...",7.2,219,NaN,NaN,NaN,Ekaa Art Production
tt9905412,Ottam,2019,120,"Set in Trivandrum, the story of Ottam unfolds ...",7.8,510,INR 4000000,NaN,$ 4791,Thomas Thiruvalla Films
tt9905462,Pengalila,2019,111,An unusual bond between a sixty year old Dalit...,8.4,604,INR 10000000,NaN,NaN,Benzy Productions
tt9911774,Padmavyuhathile Abhimanyu,2019,130,NaN,8.4,369,NaN,NaN,NaN,RMCC Productions
tt9914286,Sokagin Çocuklari,2019,98,NaN,7.2,190,NaN,NaN,$ 2833,Gizem Ajans


In [61]:
#Map company name to production_company DataFrame and add column company_id
transform_movie_df['company_id'] = transform_movie_df['company_name'].map(company_df.set_index('company_name')['company_id'])

#Drop person name column to form last part of movie_person_title_junction table
final_movie = transform_movie_df.drop(columns='company_name')
final_movie.tail(20)

,movie_title,year_published,movie_duration,description,votes_avg,votes_count,budget,usa_gross_income,worlwide_gross_income,company_id
movie_id,,,,,,,,,,
tt9860728,Falling Inn Love,2019,98,When city girl Gabriela spontaneously enters a...,5.6,9816,NaN,NaN,NaN,42
tt9860860,Abduction 101,2019,77,Three beautiful women find a strange house in ...,2.4,146,$ 500000,NaN,NaN,29137
tt9861522,Ali,2019,110,Ali's illness not only changes his life but al...,5.0,119,NaN,NaN,NaN,29926
tt9866208,Beyond the Line,2019,78,It's the close of World War II. Unwilling to s...,3.6,150,NaN,NaN,NaN,30113
tt9866700,Paranormal Investigation,2018,92,When a young man becomes possessed after playi...,3.7,1145,NaN,NaN,NaN,30114
tt9870726,Gholamreza Takhti,2019,113,The story of life of Gholam Reza Takhti from h...,6.7,192,NaN,NaN,NaN,7055
tt9872556,Momenti di trascurabile felicità,2019,93,"After a deadly accident, Paolo comes back on E...",6.3,413,NaN,NaN,$ 2057075,14751
tt9875852,Domovoy,2019,90,A single mother buys a flat in the Soviet high...,5.1,103,NaN,NaN,$ 1842316,42
tt9878242,Subharathri,2019,130,A simple love story between a man and his wife...,6.1,148,INR 30000000,NaN,$ 105932,17170


# LOAD

Create tables in pgAdmin prior to loading them

## Create database connection

In [62]:
connection_string = f'{username}:{password}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

In [63]:
# Confirm tables
engine.table_names()

['movie_language_junction',
 'language',
 'production_company',
 'movie',
 'movie_person_title_junction',
 'person',
 'job_title',
 'movie_genre_junction',
 'genre',
 'movie_country_junction',
 'country_origin']

### Load DataFrames into database

In [64]:
company_df.to_sql(name='production_company', con=engine, if_exists='append', index=False)

In [65]:
#Load movie DF into SQL
final_movie.to_sql(name='movie', con=engine, if_exists='append', index=True)

In [66]:
#Load person DF into SQL
final_name.to_sql(name='person', con=engine, if_exists='append', index=False)

In [67]:
#Load job_title DF into SQL
job_title_df.to_sql(name='job_title', con=engine, if_exists='append', index=False)

In [68]:
#Load movie_person_title_junction DF into SQL
final_mpt_junction.to_sql(name='movie_person_title_junction', con=engine, if_exists='append', index=False)

In [69]:
# Load country_origin DF into SQL
country_origin_df.to_sql(name='country_origin', con=engine, if_exists='append', index=False)

In [70]:
# Load movie_country_junction DF into SQL
movie_country_junction_df.to_sql(name='movie_country_junction', con=engine, if_exists='append', index=False)

In [71]:
# Load language DF into SQL
language_df.to_sql(name='language', con=engine, if_exists='append', index=False)

In [72]:
# Load movie_language_junction DF into SQL
movie_language_junction_df.to_sql(name='movie_language_junction', con=engine, if_exists='append', index=False)

In [73]:
# Load genre DF into SQL
genre_df.to_sql(name='genre', con=engine, if_exists='append', index=False)

In [74]:
# Load movie_genre_junction DF into SQL
movie_genre_junction_df.to_sql(name='movie_genre_junction', con=engine, if_exists='append', index=False)